In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import locale
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Configurando o idioma para português do Brasil
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')

### Funções

In [ ]:
# Função para obter a estação do ano com base no mês
def obter_estacao(do_mes):
    if 3 <= do_mes <= 5:
        return 'Outono'
    elif 6 <= do_mes <= 8:
        return 'Inverno'
    elif 9 <= do_mes <= 11:
        return 'Primavera'
    else:
        return 'Verão'
    
def gerar_consumo(**kargs) -> pd.DataFrame:
    num_linhas = kargs.get('num_linhas', 100)
    data_inicial = kargs.get('data_inicial', datetime.now())
    data_atual = datetime(data_inicial.year, data_inicial.month, data_inicial.day, data_inicial.hour) 
    quantidade_de_pessoas = kargs.get('quantidade_de_pessoas', 3)
    
    np.random.seed(datetime.now().microsecond)
    
    # Listas para armazenar os dados gerados
    anos, meses, dias_da_semana, dias_do_mes, horas = [], [], [], [], [],
    estacoes, qtd_pessoas, consumo_agua = [], [], []

    # Loop para gerar dados
    for _ in range(num_linhas):
        anos.append(data_atual.year)
        meses.append(data_atual.month)
        dias_da_semana.append(data_atual.strftime('%A'))
        dias_do_mes.append(data_atual.day)
        horas.append(data_atual.hour)
        estacoes.append(obter_estacao(data_atual.month))
        qtd_pessoas.append(quantidade_de_pessoas)
        
        if (estacoes[-1] == 'Verão'):
            if (dias_da_semana[-1]  == 'Sábado' or dias_da_semana[-1] == 'Domingo'):
                if (data_atual.hour >= 6 and data_atual.hour < 18):
                    consumo_agua.append(round(np.random.uniform(9, 30), 2))
                elif (data_atual.hour >= 18 and data_atual.hour < 21):
                    consumo_agua.append(round(np.random.uniform(7, 20), 2))
                else:
                    consumo_agua.append(round(np.random.uniform(0, 8), 2))
            elif (dias_da_semana[-1] == 'Sexta-feira'):
                if (data_atual.hour >= 6 and data_atual.hour < 18):
                    consumo_agua.append(round(np.random.uniform(7, 25), 2))
                elif (data_atual.hour >= 18 and data_atual.hour < 21):
                    consumo_agua.append(round(np.random.uniform(7, 30), 2))
                else:
                    consumo_agua.append(round(np.random.uniform(1, 10), 2))
            else:
                if (data_atual.hour >= 6 and data_atual.hour < 18):
                    consumo_agua.append(round(np.random.uniform(7, 20), 2))
                elif (data_atual.hour >= 18 and data_atual.hour < 21):
                    consumo_agua.append(round(np.random.uniform(6, 25), 2))
                else:
                    consumo_agua.append(round(np.random.uniform(2, 8), 2))
        else:
            if (dias_da_semana[-1]  == 'Sábado' or dias_da_semana[-1] == 'Domingo'):
                if (data_atual.hour >= 6 and data_atual.hour < 18):
                    consumo_agua.append(round(np.random.uniform(5, 25), 2))
                elif (data_atual.hour >= 18 and data_atual.hour < 21):
                    consumo_agua.append(round(np.random.uniform(3, 15), 2))
                else:
                    consumo_agua.append(round(np.random.uniform(0, 8), 2))
            elif (dias_da_semana[-1] == 'Sexta-feira'):
                if (data_atual.hour >= 6 and data_atual.hour < 18):
                    consumo_agua.append(round(np.random.uniform(3, 15), 2))
                elif (data_atual.hour >= 18 and data_atual.hour < 21):
                    consumo_agua.append(round(np.random.uniform(5, 20), 2))
                else:
                    consumo_agua.append(round(np.random.uniform(0, 8), 2))
            else:
                if (data_atual.hour >= 6 and data_atual.hour < 18):
                    consumo_agua.append(round(np.random.uniform(5, 15), 2))
                elif (data_atual.hour >= 18 and data_atual.hour < 21):
                    consumo_agua.append(round(np.random.uniform(5, 20), 2))
                else:
                    consumo_agua.append(round(np.random.uniform(0, 8), 2))
            
        # Avança para a próxima hora
        data_atual += timedelta(hours=1)
        
    # Criando um DataFrame com os dados gerados
    dados = pd.DataFrame({
        'ano': anos,
        'mes': meses,
        'dia_semana': dias_da_semana,
        'dia_mes': dias_do_mes,
        'hora': horas,
        'estacao': estacoes,
        'quantidade_pessoas': qtd_pessoas,
        'consumo_agua_m3': consumo_agua
    })

    return dados

### Gerar dados

In [ ]:
# Dados completos com 35040 linhas
dados = gerar_consumo(num_linhas=35040, data_inicial=datetime(2020, 1, 1), quantidade_de_pessoas=3)
dados.head()

### Agregar dados por ano e mês

In [ ]:
# Dados agregados por ano e mês
dados_agregados = dados.groupby(['ano', 'mes']).agg({'consumo_agua_m3': 'sum'}).reset_index()
dados_agregados.head()

### Visualizar dados de 2022 e 2023

In [ ]:
dados_ano_2022 = dados_agregados[dados_agregados['ano'] == 2022]
dados_ano_2023 = dados_agregados[dados_agregados['ano'] == 2023]

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].set_title(f'Consumo de água por mês em 2022')
sns.barplot(data=dados_ano_2022, x='mes', y='consumo_agua_m3', ax=axes[0])
axes[0].set_xlabel('Mês')
axes[0].set_ylabel('Consumo de água (m³)')

axes[1].set_title(f'Consumo de água por mês em 2023')
sns.barplot(data=dados_ano_2023, x='mes', y='consumo_agua_m3', ax=axes[1])
axes[1].set_xlabel('Mês')
axes[1].set_ylabel('Consumo de água (m³)')

plt.show()

### Gerar Rótulos

##### Função para gerar rotulos

In [ ]:
def pegar_dados_com_rotulo_consumo(
    dados: pd.DataFrame, gerar_outliers=False
) -> pd.DataFrame:
    copia_dados = dados.copy(deep=True)
    groupby_list = ["mes", "dia_semana", "hora"]
    dados_agrupados = (
        copia_dados.groupby(groupby_list)
        .agg({"consumo_agua_m3": ["mean", "std"]})
        .reset_index()
    )
    rotulos = []

    for _, value in copia_dados.iterrows():
        # Pega o mês, dia da semana, hora e consumo
        mes, dia_semana, hora = value["mes"], value["dia_semana"], value["hora"]
        consumo = value["consumo_agua_m3"]

        # Pegando a média e o desvio padrão de consumo de água baseado no mês, dia da semana e hora
        media_consumo_m3 = dados_agrupados[
            (dados_agrupados["mes"] == mes)
            & (dados_agrupados["dia_semana"] == dia_semana)
            & (dados_agrupados["hora"] == hora)
        ]["consumo_agua_m3"]["mean"].values[0]
        std_consumo_m3 = dados_agrupados[
            (dados_agrupados["mes"] == mes)
            & (dados_agrupados["dia_semana"] == dia_semana)
            & (dados_agrupados["hora"] == hora)
        ]["consumo_agua_m3"]["std"].values[0]

        # Pega o limite superior, limiar para alerta de consumo de água.
        limite_superior_consumo = std_consumo_m3 * 2 + media_consumo_m3

        # Verifica se o consumo é normal ou alto
        if consumo > limite_superior_consumo:
            rotulos.append("Alto")
        else:
            # Se gerar_outliers for verdadeiro, gera outliers aleatórios
            if gerar_outliers:
                np.random.seed(datetime.now().microsecond)
                random_number = np.random.randint(1, 3)

                if random_number == 1:
                    rotulos.append("Alto")
                    novo_consumo = round(
                        np.random.uniform(
                            limite_superior_consumo * 1.5,
                            limite_superior_consumo * 2.5,
                        ),
                        2,
                    )
                    copia_dados.at[_, "consumo_agua_m3"] = novo_consumo
                else: 
                    rotulos.append("Normal")
            else:
                rotulos.append("Normal")

    copia_dados["padrao_consumo"] = rotulos

    return copia_dados

##### Gerar e visualizar rótulos

In [ ]:
dados_com_rotulo = pegar_dados_com_rotulo_consumo(dados, gerar_outliers=True)
dados_com_rotulo.head()

##### Visualizar distribuição de frequência

In [ ]:
dados_com_rotulo['padrao_consumo'].value_counts()

##### Salvar dados

In [ ]:
dados_com_rotulo.to_csv('../data/dados_consumo_agua.csv', index=False)